# Тема “POS-tagger и NER”

## Задание 1. Написать теггер на данных с русским языком
проверить UnigramTagger, BigramTagger, TrigramTagger и их комбмнации
написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов
сравнить все реализованные методы сделать выводы
 
## Задание 2. Проверить насколько хорошо работает NER
данные брать из http://www.labinform.ru/pub/named_entities/
проверить NER из nltk/spacy/deeppavlov
написать свой нер попробовать разные подходы
передаём в сетку токен и его соседей
передаём в сетку только токен
свой вариант
сравнить ваши реализованные подходы на качество (вывести precision/recall/f1_score)




## Теггер на данных с русским языком

In [1]:
import nltk

from nltk.tag import DefaultTagger, UnigramTagger, BigramTagger, TrigramTagger, RegexpTagger

### Data downloading and processing

https://github.com/pyconll/pyconll



In [2]:
!pip install pyconll -q

In [3]:
#нужно обрабатывать xml
#!wget http://opencorpora.org/files/export/annot/annot.opcorpora.xml.zip -q
!wget -O ru_syntagrus-ud-train.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-c.conllu -q
!wget -O ru_syntagrus-ud-dev.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu -q

In [4]:
import pyconll

train = pyconll.load_from_file('ru_syntagrus-ud-train.conllu')
test = pyconll.load_from_file('ru_syntagrus-ud-dev.conllu')

In [5]:
# various token fields
sent_n = 0
token_n = 1
train[sent_n][token_n].id, train[sent_n][token_n].form, train[sent_n][token_n].lemma, train[sent_n][token_n].upos, train[sent_n][token_n].xpos,
#[sent_n][token_n].misc, train[sent_n][token_n].head, train[sent_n][token_n].feats, train[sent_n][token_n].deps

('2', 'мать', 'мать', 'NOUN', None)

In [6]:
def conllu_data_extraction(conllu_sentences):
  sentences = []
  tokens = []
  for sent in conllu_sentences:
    temp_t = []
    temp_s = []
    for token in sent:
      temp_t.append(tuple([token.lemma,token.upos]))
      temp_s.append(token.lemma)
    sentences.append(temp_s)
    tokens.append(temp_t)
  return tokens, sentences

In [7]:
test_sent_n = 43
train_data, _ = conllu_data_extraction(train)
test_data, test_sent = conllu_data_extraction(test)
test_sent = test_sent[test_sent_n]
test_data[test_sent_n]

[('такой', 'DET'),
 ('образ', 'NOUN'),
 (',', 'PUNCT'),
 ('мы', 'PRON'),
 ('видеть', 'VERB'),
 (',', 'PUNCT'),
 ('что', 'SCONJ'),
 ('латинизировать', 'VERB'),
 ('имя', 'NOUN'),
 ('среднеазиатский', 'ADJ'),
 ('ученый', 'NOUN'),
 ('быть', 'AUX'),
 ('вынести', 'VERB'),
 ('в', 'ADP'),
 ('заглавие', 'NOUN'),
 ('книга', 'NOUN'),
 (',', 'PUNCT'),
 ('и', 'CCONJ'),
 ('сегодня', 'ADV'),
 ('ни', 'PART'),
 ('у', 'ADP'),
 ('кто', 'PRON'),
 ('нет', 'VERB'),
 ('сомнение', 'NOUN'),
 (',', 'PUNCT'),
 ('что', 'SCONJ'),
 ('слово', 'NOUN'),
 ('"', 'PUNCT'),
 ('алгоритм', 'NOUN'),
 ('"', 'PUNCT'),
 ('попасть', 'VERB'),
 ('в', 'ADP'),
 ('европейский', 'ADJ'),
 ('язык', 'NOUN'),
 ('именно', 'PART'),
 ('благодаря', 'ADP'),
 ('этот', 'DET'),
 ('сочинение', 'NOUN'),
 ('.', 'PUNCT')]

### Unigram Tegger

In [8]:
unigram_tagger = UnigramTagger(train_data)

In [9]:
display(unigram_tagger.tag(test_sent), unigram_tagger.evaluate(test_data))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  """Entry point for launching an IPython kernel.


[('такой', 'DET'),
 ('образ', 'NOUN'),
 (',', 'PUNCT'),
 ('мы', 'PRON'),
 ('видеть', 'VERB'),
 (',', 'PUNCT'),
 ('что', 'SCONJ'),
 ('латинизировать', None),
 ('имя', 'NOUN'),
 ('среднеазиатский', None),
 ('ученый', 'NOUN'),
 ('быть', 'AUX'),
 ('вынести', 'VERB'),
 ('в', 'ADP'),
 ('заглавие', None),
 ('книга', 'NOUN'),
 (',', 'PUNCT'),
 ('и', 'CCONJ'),
 ('сегодня', 'ADV'),
 ('ни', 'PART'),
 ('у', 'ADP'),
 ('кто', 'PRON'),
 ('нет', 'VERB'),
 ('сомнение', 'NOUN'),
 (',', 'PUNCT'),
 ('что', 'SCONJ'),
 ('слово', 'NOUN'),
 ('"', 'PUNCT'),
 ('алгоритм', 'NOUN'),
 ('"', 'PUNCT'),
 ('попасть', 'VERB'),
 ('в', 'ADP'),
 ('европейский', 'ADJ'),
 ('язык', 'NOUN'),
 ('именно', 'PART'),
 ('благодаря', 'ADP'),
 ('этот', 'DET'),
 ('сочинение', 'NOUN'),
 ('.', 'PUNCT')]

0.9038348850836643

### BigramTagger

In [10]:
bigram_tagger = BigramTagger(train_data, backoff=unigram_tagger)
display(bigram_tagger.tag(test_sent), bigram_tagger.evaluate(test_data))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  


[('такой', 'DET'),
 ('образ', 'NOUN'),
 (',', 'PUNCT'),
 ('мы', 'PRON'),
 ('видеть', 'VERB'),
 (',', 'PUNCT'),
 ('что', 'SCONJ'),
 ('латинизировать', None),
 ('имя', 'NOUN'),
 ('среднеазиатский', None),
 ('ученый', 'NOUN'),
 ('быть', 'AUX'),
 ('вынести', 'VERB'),
 ('в', 'ADP'),
 ('заглавие', None),
 ('книга', 'NOUN'),
 (',', 'PUNCT'),
 ('и', 'CCONJ'),
 ('сегодня', 'ADV'),
 ('ни', 'PART'),
 ('у', 'ADP'),
 ('кто', 'PRON'),
 ('нет', 'VERB'),
 ('сомнение', 'NOUN'),
 (',', 'PUNCT'),
 ('что', 'SCONJ'),
 ('слово', 'NOUN'),
 ('"', 'PUNCT'),
 ('алгоритм', 'NOUN'),
 ('"', 'PUNCT'),
 ('попасть', 'VERB'),
 ('в', 'ADP'),
 ('европейский', 'ADJ'),
 ('язык', 'NOUN'),
 ('именно', 'PART'),
 ('благодаря', 'ADP'),
 ('этот', 'DET'),
 ('сочинение', 'NOUN'),
 ('.', 'PUNCT')]

0.9073507389804024

### TrigramTagger

In [11]:
trigram_tagger = TrigramTagger(train_data, backoff=bigram_tagger)
display(trigram_tagger.tag(test_sent), trigram_tagger.evaluate(test_data))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  


[('такой', 'DET'),
 ('образ', 'NOUN'),
 (',', 'PUNCT'),
 ('мы', 'PRON'),
 ('видеть', 'VERB'),
 (',', 'PUNCT'),
 ('что', 'SCONJ'),
 ('латинизировать', None),
 ('имя', 'NOUN'),
 ('среднеазиатский', None),
 ('ученый', 'NOUN'),
 ('быть', 'AUX'),
 ('вынести', 'VERB'),
 ('в', 'ADP'),
 ('заглавие', None),
 ('книга', 'NOUN'),
 (',', 'PUNCT'),
 ('и', 'CCONJ'),
 ('сегодня', 'ADV'),
 ('ни', 'PART'),
 ('у', 'ADP'),
 ('кто', 'PRON'),
 ('нет', 'VERB'),
 ('сомнение', 'NOUN'),
 (',', 'PUNCT'),
 ('что', 'SCONJ'),
 ('слово', 'NOUN'),
 ('"', 'PUNCT'),
 ('алгоритм', 'NOUN'),
 ('"', 'PUNCT'),
 ('попасть', 'VERB'),
 ('в', 'ADP'),
 ('европейский', 'ADJ'),
 ('язык', 'NOUN'),
 ('именно', 'PART'),
 ('благодаря', 'ADP'),
 ('этот', 'DET'),
 ('сочинение', 'NOUN'),
 ('.', 'PUNCT')]

0.9067582524903965

### regexp

In [12]:
patterns = [
    (r'.*ть$', 'VERB'),                # verb
    (r'.*ться?$', 'VERB'),                # verb
    (r'.*ти?$', 'VERB'),                # verb
    (r'^и{1}$', 'CCONJ'),
    (r'^или{1}$', 'CCONJ'),
    (r'.*ий{1}$', 'ADJ'),                # adjective
    (r'.*ый{1}$', 'ADJ'),                # adjective
    #(r'?\D', 'ADP'),                # proposal
    (r'^-?[0-9]+(\.[0-9]+)?$', 'CD'),  # cardinal numbers
    (r'[!\'\"();:,.?]?$', 'PUNCT'),
    (r'^[A-ZА-Я]{1}', 'PROPN'),
    (r'^[a-z]*$', 'X'),
    (r'.*', 'NOUN')                      # nouns (default)
]
regexp_tagger = RegexpTagger(patterns)
display(regexp_tagger.tag(test_sent), regexp_tagger.evaluate(test_data))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.


[('такой', 'NOUN'),
 ('образ', 'NOUN'),
 (',', 'PUNCT'),
 ('мы', 'NOUN'),
 ('видеть', 'VERB'),
 (',', 'PUNCT'),
 ('что', 'NOUN'),
 ('латинизировать', 'VERB'),
 ('имя', 'NOUN'),
 ('среднеазиатский', 'ADJ'),
 ('ученый', 'ADJ'),
 ('быть', 'VERB'),
 ('вынести', 'VERB'),
 ('в', 'NOUN'),
 ('заглавие', 'NOUN'),
 ('книга', 'NOUN'),
 (',', 'PUNCT'),
 ('и', 'CCONJ'),
 ('сегодня', 'NOUN'),
 ('ни', 'NOUN'),
 ('у', 'NOUN'),
 ('кто', 'NOUN'),
 ('нет', 'VERB'),
 ('сомнение', 'NOUN'),
 (',', 'PUNCT'),
 ('что', 'NOUN'),
 ('слово', 'NOUN'),
 ('"', 'PUNCT'),
 ('алгоритм', 'NOUN'),
 ('"', 'PUNCT'),
 ('попасть', 'VERB'),
 ('в', 'NOUN'),
 ('европейский', 'ADJ'),
 ('язык', 'NOUN'),
 ('именно', 'NOUN'),
 ('благодаря', 'NOUN'),
 ('этот', 'VERB'),
 ('сочинение', 'NOUN'),
 ('.', 'PUNCT')]

0.6284979490852269

### Combined tegger Trigram on regexp

In [13]:
trigram_tagger = TrigramTagger(train_data, backoff=regexp_tagger)
display(trigram_tagger.tag(test_sent), trigram_tagger.evaluate(test_data))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  


[('такой', 'DET'),
 ('образ', 'NOUN'),
 (',', 'PUNCT'),
 ('мы', 'PRON'),
 ('видеть', 'VERB'),
 (',', 'PUNCT'),
 ('что', 'SCONJ'),
 ('латинизировать', 'VERB'),
 ('имя', 'NOUN'),
 ('среднеазиатский', 'ADJ'),
 ('ученый', 'NOUN'),
 ('быть', 'AUX'),
 ('вынести', 'VERB'),
 ('в', 'ADP'),
 ('заглавие', 'NOUN'),
 ('книга', 'NOUN'),
 (',', 'PUNCT'),
 ('и', 'CCONJ'),
 ('сегодня', 'ADV'),
 ('ни', 'PART'),
 ('у', 'ADP'),
 ('кто', 'PRON'),
 ('нет', 'VERB'),
 ('сомнение', 'NOUN'),
 (',', 'PUNCT'),
 ('что', 'SCONJ'),
 ('слово', 'NOUN'),
 ('"', 'PUNCT'),
 ('алгоритм', 'NOUN'),
 ('"', 'PUNCT'),
 ('попасть', 'VERB'),
 ('в', 'ADP'),
 ('европейский', 'ADJ'),
 ('язык', 'NOUN'),
 ('именно', 'PART'),
 ('благодаря', 'ADP'),
 ('этот', 'DET'),
 ('сочинение', 'NOUN'),
 ('.', 'PUNCT')]

0.9137508952405755

## NER

In [14]:
# 2 файла - текст и соответствие текст-тег
!wget http://www.labinform.ru/pub/named_entities/collection5.zip -q

In [15]:
!unzip -q collection5.zip 

#### Data preparation

In [16]:
def file_decomposition(text):
  temp = text.split(f'\n')
  result = []
  for line in temp:
    result.append(line.split()[1:5:3]) 
  return result

def del_duplicates(current_list):
  result = []
  for i in current_list:
    if not(i in result):
      result.append(i)
  return result

def file_data_extraction(files_list, tags_read = True):
  result = []
  if tags_read:
    for filename in files_list:
      with open(os.path.join(os.getcwd(), filename), 'r') as f:
        text = f.read()
        result.append(file_decomposition(text)[:][:])
        result = del_duplicates(result)
  else:
    for filename in files_list:
      with open(os.path.join(os.getcwd(), filename), 'r') as f:
        text = f.read()
        result.append(text)
  return result

def check_text_data_files(data_file_names):
  import os
  result_text_filenames = []
  result_data_filenames = []
  for name in data_file_names:
    if os.path.exists(os.path.join(os.getcwd(), name[:-3]+'txt')):
      result_text_filenames.append(name[:-3]+'txt')
      result_data_filenames.append(name)
  return result_data_filenames, result_text_filenames

In [17]:
import os
import glob

#number of documents for train set
documents_number = 500
# number of test documents 
test_n = 2
c = 1
train_names = glob.glob('Collection5/*.ann')[0:documents_number]
# need to check an order
#train_text_names = glob.glob('Collection5/*.txt')[0:documents_number]
train_names, train_text_names = check_text_data_files(train_names)

test_names = glob.glob('Collection5/*.ann')[documents_number:documents_number+test_n]
# need to check an order
#test_text_names = glob.glob('Collection5/*.txt')[0:documents_number]
test_names, test_text_names = check_text_data_files(test_names)

train_text = file_data_extraction(train_text_names, tags_read=False)
train_set = file_data_extraction(train_names)
test_text = file_data_extraction(test_text_names, tags_read=False)
test_set = file_data_extraction(test_names)

In [88]:
def list_conversion(data):
  result = []
  for list_item in data:
    for item in list_item:
      if item:
        result.append(item)
  return result

In [ ]:
import random

sample_size = 10
start_index = random.randrange(len(train_text) - sample_size-1)
text = ' '.join(train_text[start_index:start_index+sample_size])
testsets = list_conversion(train_set[start_index:start_index+sample_size])

#### NLTK

In [20]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

ORGANIZATION	Georgia-Pacific Corp., WHO

PERSON	Eddy Bonte, President Obama

LOCATION	Murray River, Mount Everest

DATE	June, 2008-06-29

TIME	two fifty a m, 1:30 p.m.

MONEY	175 million Canadian Dollars, GBP 10.40

PERCENT	twenty pct, 18.75 %

FACILITY	Washington Monument, Stonehenge

GPE	South East Asia, Midlothian

In [90]:
refsets = [(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(text, language='russian'))) if hasattr(chunk, 'label') ]
#refsets = [item[1] for item in refsets]

In [93]:
print(f'Initial set length: {len(testsets)}, nltk NER set length: {len(refsets)}')

Initial set length: 180, nltk NER set length: 70


Due to huge difference in initial NER data and data from nltk and some errors in initial NER data it is impossible to correctly anilize metrics!!!

#### ntlk metrics on test data

In [95]:
from nltk.metrics.scores import (precision, recall)
from nltk.metrics import *

In [97]:
reference = 'DET NN VB DET JJ NN NN IN DET NN'.split()
test    = 'DET VB VB DET NN NN NN IN DET NN'.split()
print(accuracy(reference, test))

reference_set = set(reference)
test_set = set(test)
print(precision(reference_set, test_set))
print(recall(reference_set, test_set))
print(f_measure(reference_set, test_set))

0.8
1.0
0.8
0.8888888888888888


#### Spacy

In [40]:
!pip -q install spacy

In [38]:
import spacy

In [39]:
!spacy download ru_core_news_md -q

2022-07-29 10:49:07.134973: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
     |████████████████████████████████| 41.9 MB 1.2 MB/s 
     |████████████████████████████████| 55 kB 2.0 MB/s 
     |████████████████████████████████| 8.2 MB 11.4 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_md')


In [42]:
from spacy import displacy
import ru_core_news_md

##### Displaying NER

In [98]:
nlp = ru_core_news_md.load()
article = nlp(text)
displacy.render(article, jupyter=True, style='ent')

##### Just NER data

In [116]:
refsets = [[item.ent_type_, item.text] for item in article]

In [127]:
refsets = [item for item in refsets if item[0]]
list(zip(testsets, refsets))[:20]

[(['PER', 'С.Собяниным'], ['PER', 'С.Собяниным']),
 (['PER', 'Сергеем'], ['PER', '\n\n']),
 (['LOC', 'Москвы'], ['PER', 'Вслед']),
 (['GEOPOLIT', 'России'], ['PER', 'Сергеем']),
 (['PER', 'Владимир'], ['PER', 'Собяниным']),
 (['PER', 'С.Собянин'], ['LOC', 'Москвы']),
 (['LOC', 'Москвы'], ['LOC', 'России']),
 (['PER', 'В.Путин'], ['PER', 'Владимир']),
 (['GEOPOLIT', 'России'], ['PER', 'Путин']),
 (['LOC', 'Подмосковья'], ['PER', 'С.Собянин']),
 (['PER', 'Сергея'], ['LOC', 'Москвы']),
 (['PER', 'Юрий'], ['LOC', 'В.Путин']),
 (['PER', 'Михаила'], ['LOC', 'России']),
 (['PER', 'Алексея'], ['LOC', 'Подмосковья']),
 (['LOC', 'Москвы'], ['PER', 'Сергея']),
 (['PER', 'В.Путин'], ['PER', 'Собянина']),
 (['GEOPOLIT', 'Абхазии'], ['PER', 'Юрий']),
 (['PER', 'Л.Лакербая'], ['PER', 'Лужков']),
 (['ORG', 'правительства'], ['PER', 'Михаила']),
 (['PER', 'Владимир'], ['PER', 'Прохорова'])]

In [125]:
len(refsets), len(testsets)

(273, 180)

В сравнении с nltk, spacy "из коробки" работает намного лучше и определяет даже те именованные сущности, которые в предобработанных данных не были учтены. Но опять не имеет смысла считать метрики, так как исходные данные некорректны!